# Getting Started Notebook: Stage 2

This notebook illustrates how to use the datasets provided to you and will be required to submit your first model.
This notebook follows the same structure as the stage 1 notebook, but you will need to perform some additional steps such as feature extraction.

## 0. Dependencies

This notebook requires several Python **3** packages, which are included in Anaconda 3 for Python 3.8, which is the python distribution we recommend you to use throughout this course.
The package versions listed below have been used for testing and are confirmed to work well.
We strongly recommend you to install these specific versions to ensure this notebook works as expected and we can offer you optimal support throughout the competition.

python: 3.8

scikit-learn: 1.0.0

numpy: 1.20.1

matplotlib: 3.4.3

## 1. How to use this Notebook

This Notebook is based on the stage 1 notebook and provides a sample structure to load data, extract features, and train and tune a scikit-learn pipeline for stage 2.
You are not required to follow this exact structure.
After you have completed all necessary steps, this notebook will generate a CSV (.csv) file to be submitted on the [Kaggle competition page](https://www.kaggle.com/c/ugentml21-slc-2/)

In [ ]:
!pip install scikit-learn==1.0.0 numpy matplotlib

In [ ]:
# your data, used to name the output file
student_id = "02010068"
student_lastname = "Dagrain" 
student_firstname = "Miguel"

# change this if you would like your submission outputfile to have a more detailed name, e.g. submission_with_special_preprocessing 
submission_prefix='submission'

# whether or not you want your created models and submissions versioned using timestamps
# (setting this to False will overwrite previously exported model and submission files of the same name)
use_timestamps = True

## 2. Loading the data

The dataset contains videos of people signing in Flemish sign language (Vlaamse Gebarentaal). It consists of 15 classes corresponding to lexical signs. From these videos, 3D keypoints were extracted using MediaPipe Holistic. In total, there are 125 keypoints, resulting in 375 (=3x125) floating point values per video frame.

In this stage, we expect that you perform your own feature engineering. As a baseline, you can start by extracting the same set of features as we did for stage 1.
You can then compare this and future models with the baseline on the Kaggle competition page.

As a reminder: the set of features in stage 1 were the time averages of all keypoint coordinates over the first and the second half of the sample frames, so 750 features in total.

We start by importing the libraries we need: 
- sklearn and numpy to do machine learning, 
- csv and pickle read the data and write out submission and model files, 
- time and os to keep organized with the files we output,
- matplotlib to perform visualizations.
We also import some specific sklearn components as well as an utils library with some handy extra functions.

In [ ]:
import sklearn
import numpy as np
import csv
import pickle
import time
import os
import matplotlib.pyplot as plt

%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedGroupKFold, GridSearchCV

from utils_general import utils_for_students

We can use our utils_for_students library to load the data from disk. Remember to put the [unzipped files from the competition page](https://www.kaggle.com/c/ugentml21-slc-2/data) into the right paths on your filesystem.

**Note: stage 2 uses different data files than stage 1!**

In [ ]:
train_samples = utils_for_students.load_dataset_stage2('data/stage2_labels_train.csv', 'train')
test_samples = utils_for_students.load_dataset_stage2('data/stage2_ids_test.csv', 'test')

For the train data, we get a list of python dictionaries, where each dictionary corresponds to one sign language clip. The dictionary has the following keys:

In [ ]:
train_samples[0].keys()

For the test data, we also get a list of python dictionaries, but with fewer keys:

In [ ]:
test_samples[0].keys()

You can load an individual sample using the provided code.

In [ ]:
sample = train_samples[0]

landmark_sequence = utils_for_students.load_sample_stage2(os.path.join('data/stage2/train/', sample['path']))

We can use the utility code to create a visualization of individual video frames of every sign instance.

In [ ]:
FRAME_INDEX = 0

utils_for_students.visualize_pose(landmark_sequence[FRAME_INDEX], plt)

In order to get a better grasp of every class in the dataset, we provide some additional information here.

There are 15 different classes. Each class corresponds to a sign. Signs can be annotated using sign glosses, which are representations of signs in written form in a spoken language. In the below table, we list all glosses, the English meaning of the corresponding sign, and provide a link to the Flemish sign language dictionary so that you can see an example performance of that sign.

| Gloss             | English meaning | Dictionary URL                                                                                        |
|-------------------|-----------------|-------------------------------------------------------------------------------------------------------|
| c.AF              | Done, finished  | https://vlaamsegebarentaal.be/signbank/dictionary/protected_media/glossvideo/AF/AF-B-209.mp4          |
| c.OOK             | Too, also       | https://vlaamsegebarentaal.be/signbank/dictionary/protected_media/glossvideo/OO/OOK-A-8491.mp4        |
| ZELFDE-A          | The same        | https://vlaamsegebarentaal.be/signbank/dictionary/protected_media/glossvideo/ZE/ZELFDE-A-14290.mp4    |
| AUTO-RIJDEN-A     | To drive a car  | https://vlaamsegebarentaal.be/signbank/dictionary/protected_media/glossvideo/RI/RIJDEN-C-9982.mp4     |
| HEBBEN-A          | To have         | https://vlaamsegebarentaal.be/signbank/dictionary/protected_media/glossvideo/HE/HEBBEN-A-4801.mp4     |
| HAAS-oor          | Hare            | https://vlaamsegebarentaal.be/signbank/dictionary/protected_media/glossvideo/HA/HAAS-B-16147.mp4      |
| AANKOMEN-A        | To arrive       | https://vlaamsegebarentaal.be/signbank/dictionary/protected_media/glossvideo/AA/AANKOMEN-A-39.mp4     |
| SCHILDPAD-Bhanden | Turtle          | https://vlaamsegebarentaal.be/signbank/dictionary/protected_media/glossvideo/SC/SCHILDPAD-A-10503.mp4 |
| WAT-A             | What            | https://vlaamsegebarentaal.be/signbank/dictionary/protected_media/glossvideo/WA/WAT-A-13657.mp4       |
| c.ZIEN            | To see          | https://vlaamsegebarentaal.be/signbank/dictionary/protected_media/glossvideo/BE/BEKIJKEN-A-1157.mp4   |
| NAAR-A            | Towards         | https://vlaamsegebarentaal.be/signbank/dictionary/protected_media/glossvideo/GA/GA-NAAR-A-4032.mp4    |
| MOETEN-A          | To must         | https://vlaamsegebarentaal.be/signbank/dictionary/protected_media/glossvideo/MO/MOETEN-D-17652.mp4    |
| C: 1              | 1               | https://vlaamsegebarentaal.be/signbank/dictionary/protected_media/glossvideo/1-/1-A-15277.mp4         |
| GOED-A            | Good            | https://vlaamsegebarentaal.be/signbank/dictionary/protected_media/glossvideo/GO/GOED-C-4413.mp4       |
| C: 2              | 2               | https://vlaamsegebarentaal.be/signbank/dictionary/protected_media/glossvideo/2-/2-A-15278.mp4         |

## 3. Feature Extraction

For stage 1, we performed feature extraction for you.

In this stage, you will need to perform feature extraction yourself.

In [ ]:
#from frame_avg import extract_features
#from sklearn.preprocessing import normalize
from utils_feature_preprocessing.split_features_into_frames import split_features_into_frames

def extract_features(pose_sequence):
    return split_features_into_frames(pose_sequence, k=3)

In [ ]:
from utils_feature_preprocessing.correct_zero_values import correct_zeros
# Concatenate the training set features.
X_train = []
y_train = []
signers_train = []
for sample in train_samples:
    pose_sequence = utils_for_students.load_sample_stage2(os.path.join('data/stage2/train/', sample['path']))
    pose_sequence = correct_zeros(pose_sequence)
    X_train.append(extract_features(pose_sequence))
    y_train.append(sample['label'])
    signers_train.append(sample['signer'])
    
# Concatenate the test set features.
X_test = []
test_ids = []
for sample in test_samples:
    pose_sequence = utils_for_students.load_sample_stage2(os.path.join('data/stage2/test/', sample['path']))
    pose_sequence = correct_zeros(pose_sequence)
    X_test.append(extract_features(pose_sequence))
    test_ids.append(sample['id'])

#Combining to numpy array
X_train = np.stack(X_train)
X_test = np.stack(X_test)

# Encode the labels as integers
label_encoder = utils_for_students.label_encoder()
y_train = label_encoder.transform(y_train)

In [ ]:
print(len(set(signers_train)))
unique_signers = set(signers_train)

In [ ]:
print(X_train.shape)
print(X_test.shape)

## 4. Creating pipelines for preprocessing and feature selection 

Now, we are ready to define our pipelines. You can create the same pipeline as for stage 1 as a baseline, but you will be expected to tune both the feature engineering
and pipeline parts of your model iteratively to improve your results.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif 
from sklearn.linear_model import RidgeClassifier
from sklearn.decomposition import PCA

preprocessing = Pipeline([
     ('scaler', StandardScaler()),
     ('decompose', PCA()),
                        ]) 

#TODO: define feature selection pipeline here
feature_selection = Pipeline([
    ('selectKBest', SelectKBest()),
                            ]) 

## 5. Define a suitable classifier
With your preprocessing and feature selection in place, it is now time to define the final element: a suitable classifier. 

In [ ]:
#TODO: define proper classifier
classifier = RidgeClassifier(class_weight = "balanced")

## 6. Set up hyperparameter grid for [GridsearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html?highlight=gridsearchcv#sklearn.model_selection.GridSearchCV) object.

Now, define your GridSearchCV hyperparameter grid and object.


In [ ]:
param_grid = {
    'preprocessing__decompose__n_components': [None],
    'preprocessing__decompose__whiten': [True, False],
    'feature_selection__selectKBest__k': [100,200,300,400,500,700,800,900,1000,1125],
    'classifier__alpha': [10, 1, 0.1, 0.01, 0.001],
    'classifier__tol': [1.0e-5, 1.0e-4,  1.0e-3],
    'classifier__fit_intercept': [True, False]
}

## 7. Define the number of crossvaldation folds and how to split

Now, you should define the number of CV folds and how to split the data.
Assuming you had a correct split in the first stage of the competition, you can re-use the same code.


In [ ]:
#apparently there are now 38 different signers
n_folds = 14

# The function below is just an example!
#TODO: write a better split function here?
#split according to signer for i.i.d. sets
def create_folds(X,y,n_folds):
    folds = []
    cv_object = StratifiedGroupKFold(n_splits = n_folds)
    for (train_indices, val_indices) in  cv_object.split(X_train, y_train, groups=signers_train):
        folds.append((train_indices,val_indices))
    return folds

## 8. Training the model
Now it is time to put everything together and train the model. As you can see, `GridsearchCV` takes the pipelines as well as the classifier and the hyperparameter dictionary you defined, and uses `create_folds` to create list of train and test indices for each split. Then the model is trained using `cv.fit()` and the model and submission files are written to the file system.

In stage 2, this cell is no longer locked and you are free to edit it as you like.

In [ ]:
pipeline = Pipeline([
    ('preprocessing', preprocessing),
    ('feature_selection', feature_selection),
    ('classifier', classifier)])

folds = create_folds(X_train,y_train,n_folds)
assert isinstance(folds,list),'Folds must be presented as tuples of train and test index lists' 

# train model
cv = GridSearchCV(pipeline, param_grid, n_jobs=4, cv=folds, verbose=10, return_train_score=True, refit=True)
cv.fit(X_train, y_train)

In [ ]:
# write out model
#make sure student data is filled in to give the file a speaking name
assert student_id is not None and student_lastname is not None and student_firstname is not None, 'Please fill in your Name and Student Id'

submission_dirname = 'submission'
if use_timestamps:
    timestamp = time.strftime("%Y%m%d-%H%M%S", time.localtime())
    filename_model = os.path.join(submission_dirname,f'stage2_model_{student_id}_{student_lastname}_{student_firstname}_{timestamp}.pkl')
    filename_submission =  os.path.join(submission_dirname,f'stage2_{submission_prefix}_{student_id}_{student_lastname}_{student_firstname}_{timestamp}.csv')
else:
    filename_model = os.path.join(submission_dirname,f'stage2_model_{student_id}_{student_lastname}_{student_firstname}.pkl')
    filename_submission =  os.path.join(submission_dirname,f'stage2_{submission_prefix}_{student_id}_{student_lastname}_{student_firstname}.csv')

if not os.path.exists(submission_dirname):
    os.mkdir(submission_dirname)    

with open(filename_model,'wb') as file:
    pickle.dump(cv,file)
    
prediction = label_encoder.inverse_transform(cv.best_estimator_.predict(X_test))
utils_for_students.create_submission_file(filename_submission, test_ids, prediction)

## 9. Printing scores
Here we simply extract a bit more information about the individual scores obtained by the classifers we trained to fit the individual folds. Maybe a few plots may be useful to better understand what your classifier is doing? 

In [ ]:
results = cv.cv_results_
mean_train_score = results['mean_train_score'][cv.best_index_]
std_train_score = results['std_train_score'][cv.best_index_]
mean_cv_score = results['mean_test_score'][cv.best_index_]
std_cv_score = results['std_test_score'][cv.best_index_]

print('Training accuracy {} +/- {}'.format(mean_train_score, std_train_score))
print('Cross-validation accuracy: {} +/- {}'.format(mean_cv_score, std_cv_score))

print('Best estimator:')
print(cv.best_estimator_)

In [ ]:
# use for visualizing certain evolutions

import matplotlib.pyplot as plt

print("Best parameters set found on development set: ",cv.best_params_)
# store the best optimization parameter for later reuse
bestC2 = cv.best_params_['classifier__alpha']

print("Grid scores on training data set:")
print()
cv_means = cv.cv_results_['mean_test_score']
cv_stds = cv.cv_results_['std_test_score']

train_means = cv.cv_results_['mean_train_score']
train_stds = cv.cv_results_['std_train_score']

C_range =[100,200,300,400,500,700,800,900,1000,1125]
plt.figure()
plt.plot(C_range,train_means,'g-',label="train")
plt.plot(C_range,cv_means,'r-',label="validate")
#plt.plot(np.log10(C_range),train_means,'g-',label="train")
#plt.plot(np.log10(C_range),cv_means,'r-',label="validate")
plt.xlabel("classifier__C")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
from utils_data_analysis.learning_curve import plot_learning_curve

fig, axes = plt.subplots(3, 1, figsize=(10, 15))
title = "Learning Curves"

plot_learning_curve(
    cv.best_estimator_, title, X_train, y_train, axes=axes, ylim=(0.0, 1.01), cv=folds, n_jobs=4
)

plt.show()

In [ ]:
from utils_general.utils_for_students import label_encoder
from sklearn.model_selection import cross_val_predict
from utils_data_analysis.confusion_matrix import plot_confusion_matrix
CLASSES = label_encoder().inverse_transform([i for i in range(len(label_encoder().classes_))])
y_pred = cross_val_predict(cv.best_estimator_, X_train, y_train, cv=folds, n_jobs=5) 
plot_confusion_matrix(y_train, y_pred, classes=CLASSES, normalize=True)